# Tfrecord

于Tensorflow读取数据，官网给出了三种方法：
* 供给数据(Feeding)： 在TensorFlow程序运行的每一步， 让Python代码来供给数据。
* 从文件读取数据： 在TensorFlow图的起始， 让一个输入管线从文件中读取数据。
* 预加载数据： 在TensorFlow图中定义常量或变量来保存所有数据(仅适用于数据量比较小的情况)。
于数据量较小而言，可能一般选择直接将数据加载进内存，然后再分batch输入网络进行训练（tip:使用这种方法时，结合yield 使用更为简洁，）。但是，如果数据量较大，这样的方法就不适用了，因为太耗内存，所以这时最好使用tensorflow提供的队列queue，也就是第二种方法 从文件读取数据。使用tensorflow内定标准格式——TFRecords。

从宏观来讲，tfrecord其实是一种数据存储形式。使用tfrecord时，实际上是先读取原生数据，然后转换成tfrecord格式，再存储在硬盘上。而使用时，再把数据从相应的tfrecord文件中解码读取出来。那么使用tfrecord和直接从硬盘读取原生数据相比到底有什么优势呢？其实，Tensorflow有和tfrecord配套的一些函数，可以加快数据的处理。实际读取tfrecord数据时，先以相应的tfrecord文件为参数，创建一个输入队列，这个队列有一定的容量（视具体硬件限制，用户可以设置不同的值），在一部分数据出队列时，tfrecord中的其他数据就可以通过预取进入队列，并且这个过程和网络的计算是独立进行的。也就是说，网络每一个iteration的训练不必等待数据队列准备好再开始，队列中的数据始终是充足的，而往队列中填充数据时，也可以使用多线程加速。

相关链接
* (https://blog.csdn.net/happyhorizion/article/details/77894055)
* (https://blog.csdn.net/u010358677/article/details/70544241)
* (https://blog.csdn.net/Best_Coder/article/details/70146441)

In [1]:
import os 
import tensorflow as tf 
from PIL import Image  
import matplotlib.pyplot as plt

把图片及标签存入tfrecord

In [13]:
tfwriter = tf.python_io.TFRecordWriter('dog_cat.tfrecords')
cwd = 'sourceImage/'
classes = ['dog','cat']
for class_ in classes:
    classpath = cwd + class_ + '/'
    for name_ in os.listdir(classpath):
        img_path = classpath + name_
        img = Image.open(img_path)
        img = img.resize((100,100))
        img_raw = img.tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={
            'label':tf.train.Feature(bytes_list = tf.train.BytesList(value=[class_.encode()])),
            'img_raw':tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_raw]))
        }))
        tfwriter.write(example.SerializeToString())
tfwriter.close()

读取函数

In [ ]:
def read_and_decode(filename):